In [14]:
import numpy as np
from pupil_apriltags import Detector
import cv2
from time import sleep, time
from IPython.display import clear_output
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
import matplotlib.pyplot as plt
%matplotlib inline

res = (1280, 760)

In [2]:
camera_info = {}
#Camera Resolution
camera_info["res"] = (1280, 760)
#Camera Intrinsic Matrix (3x3)
camera_info["K"] = np.array(
    [[631.6058624841243, 0.0, 673.9002987027918], [0.0, 627.4303222760955, 380.85431690312384], [0.0, 0.0, 1.0]])
#The non-default elements of the K array, in the AprilTag specification
camera_info["params"] = [631.605, 627.43, 673.9, 380.85]
#Fisheye Camera Distortion Matrix
camera_info["D"] = np.array(
    [[-0.031080677599846774], [-0.006061559072085696], [-0.0011641369792770276], [0.00028577486827623653]])
#Fisheye flag
camera_info["fisheye"] = True


In [4]:
families = "tagStandard52h13"
#Size of Tag in Meters
tag_size = .04
detector = Detector(families=families,quad_decimate=4.0)
import pdb;pdb.set_trace()
x=1

--Return--
None
> <ipython-input-4-5c56ed6a4a53>(5)<module>()
      2 #Size of Tag in Meters
      3 tag_size = .04
      4 detector = Detector(families=families,quad_decimate=4.0)
----> 5 import pdb;pdb.set_trace()
      6 x=1

ipdb> detector.__dict__
{'params': {'families': ['tagStandard52h13'], 'nthreads': 1, 'quad_decimate': 4.0, 'quad_sigma': 0.0, 'refine_edges': 1, 'decode_sharpening': 0.25, 'debug': 0}, 'libc': <CDLL '/home/pi/.virtualenvs/raceon/lib/python3.7/site-packages/pupil_apriltags/lib/libapriltag.so.3.1.0', handle 29b4f48 at 0xa5f17190>, 'tag_detector': None, 'tag_detector_ptr': <pupil_apriltags.bindings.LP__ApriltagDetector object at 0xa54ab440>, 'tag_families': {'tagStandard52h13': <pupil_apriltags.bindings.LP__ApriltagFamily object at 0x9d7a0620>}}
ipdb> q


BdbQuit: 

In [ ]:
import pdb;pdb.set_trace()


--Call--
> /home/pi/.virtualenvs/raceon/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 



In [29]:
def undistort(img):
        map1, map2 = cv2.fisheye.initUndistortRectifyMap(camera_info["K"], camera_info["D"], np.eye(3),
                                                         camera_info["K"], camera_info["res"], cv2.CV_16SC2)
        return cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
def undistort2(img):
    return cv2.fisheye.undistortImage(img, camera_info["K"], camera_info["D"])


In [ ]:
RUN_TIMER = 10
positions = np.zeros(shape=(3, 1))
first_frame = True
camera = PiCamera()
camera.resolution = res
camera.framerate = 10
rawCapture = PiYUVArray(camera, size=res)
chkpt1=[]
chkpt2=[]
chkpt3=[]
chkpt4=[]
t = time()
for f in camera.capture_continuous(rawCapture, format="yuv", use_video_port=True):
    t1=time()
    if first_frame:
        first_frame = False
        # Reset the buffer for the next image
        rawCapture.truncate(0)
        continue
    # Stop after RUN_TIMER seconds
    if (time() - t) > RUN_TIMER:
        break
        
    # Get the intensity component of the image (a trick to get black and white images)
    I_distorted = f.array[:, :, 0]
    rawCapture.truncate(0)
    chkpt1.append(time()-t1)
    I = undistort2(I_distorted)
    I2= undistort(I_distorted)
    chkpt2.append(time()-t1)
    detected_tags = detector.detect(I, estimate_tag_pose=True, camera_params=camera_info["params"],
                                                 tag_size=tag_size)
    chkpt3.append(time()-t1)
    tmp_poses = [tag.pose_t for tag in detected_tags]
    if tmp_poses:
        avg=np.mean(np.concatenate(tmp_poses, axis=1), axis=1, keepdims=True)
        positions = np.hstack((positions, avg))
        print(f"Time:{time() - t:.3f}, X:{avg[0,0]:.3f},Y:{avg[1,0]:.3f},X:{avg[2,0]:.3f}")
    else:
        # If no new tag detections in this frame, assume (naively) that the camera stayed in the same position
        # as the previous frame
        positions = np.hstack((positions, positions[:, -1:]))
        print("No detections")
    chkpt4.append(time()-t1)
print(positions.shape)
rawCapture.close()
camera.close()

/home/pi/.virtualenvs/raceon/lib/python3.7/site-packages/picamera/encoders.py:544: PiCameraResolutionRounded: frame size rounded up from 1280x760 to 1280x768
  width, height, fwidth, fheight)))


None
> <ipython-input-34-c2bda3f6e515>(31)<module>()
     29     I2= undistort(I_distorted)
     30     import pdb;pdb.set_trace()
---> 31     chkpt2.append(time()-t1)
     32     detected_tags = detector.detect(I, estimate_tag_pose=True, camera_params=camera_info["params"],
     33                                                  tag_size=tag_size)

ipdb> I2
array([[ 0,  0,  0, ..., 20, 20, 21],
       [ 0,  0,  0, ..., 20, 21, 22],
       [ 0,  0,  0, ..., 22, 22, 23],
       ...,
       [ 0,  0,  0, ..., 26, 25, 25],
       [ 0,  0,  0, ..., 26, 25, 24],
       [ 0,  0,  0, ..., 24, 24, 23]], dtype=uint8)
ipdb> I
array([[ 8, 32,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)
ipdb> I.shape
(760, 1280)
ipdb> I2.shape
(760, 1280)


In [31]:
camera.close()
rawCapture.close()

In [33]:
print(np.mean(chkpt1))
print(np.mean(chkpt2))
print(np.mean(chkpt3))
print(np.mean(chkpt4))

0.0015423695246378581
0.1650658349196116
0.17118163903554282
0.17161423961321512
